Create a Profile for a Layout:

City Sewer Example:
Alternatve to Profile charts in ArcGIS Pro Maps or Scenes

This script creates a vertical profile in a map by referencing XYZ data as route station/Z in the XY axis of the map projection.  Be sure to use a cartesian style projection without a distortional conic at the origin or adjust the origin from 0,0 so as not to induce a false  slope.  Do not use the KS GIS state plane projection or UTM.

Check as built designs especially for drop manholes, which only have one downstream elevation in the GIS. Drop manholes have a bottom drop of less than 2 ft per KDHE criteria. Upper drops are 2 ft or more above the invert elevation.  

Carefully QC Drop Manholes incoming conduit elevations vs inverts and slope data.  The test example has multiple issues that become evident when producing and reviewing the profile vs as-built.

To start create a map in ArcGIS Pro with sewer ssGravityMain and ssManhole layers

putting the origin at 0,0 in the map projection warps the layout, choose a cartesian coordainte system (if possible, or no coordinate system if possible) or an origin coordinate that is centered in the selected projection for minimal disortion of the profile

In [1]:
#set workspace environment and variables

ws = r"\\citydata\users\kgonterwitz\projectreviews\westdale\WestdaleSewerDesignFile.gdb"

profile = "SewerProfileWestdale"

projection = '''PROJCS["NAD_1983_2011_KS_RCS_Zone_11",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",11500000.0],PARAMETER["False_Northing",600000.0],PARAMETER["Central_Meridian",-95.25],PARAMETER["Standard_Parallel_1",39.1],PARAMETER["Scale_Factor",1.000033],PARAMETER["Latitude_Of_Origin",39.1],UNIT["Foot_US",0.3048006096012192]];-110278200 -95394100 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision'''

arcpy.env.overwriteOutput = 1

In [3]:
#select the linear infrastructure (pipe, street, etc) to profile
#translate project to projection defined above next time
#arcpy.conversion.FeatureClassToFeatureClass("ssGravityMain", ws, profile)

with arcpy.EnvManager(outputCoordinateSystem=projection):
    arcpy.conversion.FeatureClassToFeatureClass("ssGravityMain", ws, profile)

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\SewerProfileWestdale'>

In [6]:
#select the point infrastructure (manholes, inlets, etc) to profile
#translate project to projection defined above next time
#arcpy.conversion.FeatureClassToFeatureClass("ssGravityMain", ws, profile)

with arcpy.EnvManager(outputCoordinateSystem=projection):
    arcpy.conversion.FeatureClassToFeatureClass("ssManhole", ws, profile+"MH")

In [3]:
#dissolve the selected lines to profile into a single line that is the full profile length
#dissolve will keep 2 DS elevations for profiling drops
#dissolve will dissolve around maintype, dissimilar main types will not profile together

arcpy.management.Dissolve(ws+r"\\"+profile, ws+r"/"+profile+"D", "MAINTYPE", "UPELEV MAX;DOWNELEV MIN;DOWNELEV MAX;Shape_Length SUM", "MULTI_PART", "DISSOLVE_LINES", '')


<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\SewerProfileWestdaleD'>

In [4]:
#Profiles will be stationed based on upstream to downstream direction
#these steps derive coordinates for feature along the profile length
#XY coordinates are displayed as Stationing/ M domain coordiantes along the profile length
arcpy.management.CalculateField(ws+r"/"+profile+"D", "StartPoint", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.CreateRoutes(profile+"D", "MAINTYPE", ws+r"\\profileM", "TWO_FIELDS", "StartPoint", "SUM_Shape_Length", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.lr.LocateFeaturesAlongRoutes(profile, "profileM", "MAINTYPE", "0 Feet", ws+r"/profileSegments", "RID; Line; FMEAS; TMEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\profileSegments'>

In [7]:
#add manhole profiles
#review for drops and edit the vertical data for profiling manholes
arcpy.env.overwriteOutput = 1

arcpy.lr.LocateFeaturesAlongRoutes(profile+"MH", "profileM", "MAINTYPE", "0 Feet", ws+r"\\"+profile+"MHM", "RID; Point; MEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")
arcpy.management.XYTableToPoint(ws+r"\\"+profile+"MHM", ws+r"\ManholeProfileRimPt", "MEAS", "RIMELEV", None, projection)
arcpy.management.XYTableToPoint(ws+r"\\"+profile+"MHM", ws+r"\ManholeProfileInvert", "MEAS", "INVERTELEV", None, projection)
arcpy.management.Merge("ManholeProfileInvert;ManholeProfileRimPt", ws+r"\\"+profile+"MHMerge", '#', "ADD_SOURCE_INFO")

arcpy.management.PointsToLine(profile+"MHMerge", ws+r"\\"+profile+"MHProfile", "FACILITYID", "INVERTELEV", "NO_CLOSE")
arcpy.management.AddJoin(profile+"MHProfile", "FACILITYID", "ManholeProfileInvert", "FACILITYID", "KEEP_ALL", "INDEX_JOIN_FIELDS")

arcpy.management.SelectLayerByAttribute(profile+"MHProfile", "NEW_SELECTION", "ManholeProfileInvert.BARRELDIA IS NULL", None)
arcpy.management.CalculateField(profile+"MHProfile", "BufferWidth", "2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(profile+"MHProfile", "NEW_SELECTION", "ManholeProfileInvert.BARRELDIA IS NOT NULL", None)
arcpy.management.CalculateField(profile+"MHProfile", "BufferWidth", "!ManholeProfileInvert.BARRELDIA!/2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(profile+"MHProfile", "CLEAR_SELECTION")

arcpy.analysis.Buffer(profile+"MHProfile", ws+r"\\"+profile+"_Buffer", profile+"MHProfile.BufferWidth", "FULL", "FLAT", "NONE", None, "PLANAR")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\SewerProfileWestdale_Buffer'>

In [5]:
#Now we should have the station coordinates and elevation of the pipe ends for the sewer
#this cell calculates the station and elevation for the profile lines as points
#then the points can be merged and rocessed into a profile line

arcpy.management.XYTableToPoint("profileSegments", ws+r"\ProfileSegStartPt", "FMEAS", "UPELEV", None, projection)
arcpy.management.XYTableToPoint("profileSegments", ws+r"\ProfileSegEndPt", "TMEAS", "DOWNELEV", None, projection)

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\ProfileSegEndPt'>

In [14]:
#the first example was an outside drop manhole, the downstream pipe elevation and slope are based on the low connection
#the slope and profile need to show the flowline of the upper manhole
#this will have to be entered manually for now
#drop manholes have a bottom drop of less than 2 ft per KDHE criteria
#the upper drop is to meet max slope and is higher than 2 ft

#arcpy.conversion.FeatureClassToFeatureClass("ManholeProfileInvert", ws, "DropMH", "MHTYPE = 'DRP'", '#', '')
arcpy.conversion.FeatureClassToFeatureClass("ManholeProfileInvert", ws, "BuriedMH", "MHTYPE = 'Buried'", '#', '')

#DropElTop = 945.00
#DropElTSta = 118.50799-6

#DropElBottom = 935.9
#DropElBSta = 118.50799

arcpy.management.CalculateField("BuriedMH", "FMEAS", "!MEAS!-2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("BuriedMH", "UPELEV", DropElBottom, "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("BuriedMH", "DNELEV", DropElBottom, "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.XYTableToPoint(ws+r"\BuriedMH", ws+r"\BuriedMH_1", "FMEAS", "DNELEV", None, projection)


<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\BuriedMH_1'>

In [8]:
#merge the points 
arcpy.management.Merge("ProfileSegStartPt;ProfileSegEndPt;BuriedMH_1", ws+r"\Point_Merge")
arcpy.management.Sort("Point_Merge", ws+r"\Point_Merge_Sort", "FMEAS ASCENDING;UPELEV ASCENDING", "UR")
#theck the points , may need to delete a point on a drop MH

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Datasets: Dataset BuriedMH_1 does not exist or is not supported
WARNING 000725: Output Dataset: Dataset \\citydata\users\kgonterwitz\projectreviews\westdale\WestdaleSewerDesignFile.gdb\Point_Merge already exists.
Failed to execute (Merge).


In [9]:
#merge the points 
arcpy.management.Merge("ProfileSegStartPt;ProfileSegEndPt", ws+r"\Point_Merge")
arcpy.management.Sort("Point_Merge", ws+r"\Point_Merge_Sort", "FMEAS ASCENDING;UPELEV ASCENDING", "UR")
#theck the points , may need to delete a point on a drop MH

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\Point_Merge_Sort'>

In [10]:
#merge the points and make the profile lines
#check the sort and maybe delete out of order points

arcpy.management.PointsToLine("Point_Merge_Sort", ws+r"\ProfleLine", "MAINTYPE", None, "NO_CLOSE")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\ProfleLine'>

In [12]:
#version 3 create a fishnet grid, fishnet originates at upper right coordinate and nets to the left and down

#250 is the length
#965 is the top elev
#10 is hte X spacing
#1 is the Y spacing
#32 is the Y num of lines
#25 is hte X num of lines

arcpy.management.CreateFishnet(ws+r"\\profilegrid", "240 965", "250 0", 10, 1, 35, 25, None, "LABELS", '0 0 0 0', "POLYLINE")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\profilegrid'>

In [ ]:
#make a list of the outputs above and merge them to simplify labeling and symbology
arcpy.management.Merge(ProfileLines, ws+r"\ProfileLines_Merge", "#", "NO_SOURCE_INFO")

In [6]:
#create surface profile 
#DEM is used, consider using DSM/lidar for trees and built env added

DEM = r"\\gisfile\gisdata\Published\LIDAR\2021_LiDAR\Lidar_Mosaic_Dataset.gdb\QL2_DEM_2021"
arcpy.ddd.InterpolateShape(DEM, ws+r"\\"+"profileM", ws+r"\SurfaceEl", None, 1, "BILINEAR", "DENSIFY", 0, "EXCLUDE")
arcpy.management.FeatureVerticesToPoints("SurfaceEl", ws+r"\SurfacePt", "ALL")
arcpy.management.AddXY("SurfacePt")
arcpy.lr.LocateFeaturesAlongRoutes("SurfacePt", "profileM", "MAINTYPE", "0 Feet", ws+r"\profileMZ", "RID; Point; MEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")
arcpy.management.XYTableToPoint("profileMZ", ws+r"\SurfaceMZPoint", "MEAS", "POINT_Z", None, projection)
arcpy.management.PointsToLine("SurfaceMZPoint", ws+r"\SurfaceMZ", "RID", "MEAS", "NO_CLOSE")

<Result 'C:\\Users\\kgonterwitz\\OneDrive - City of Lawrence KS\\Documents\\ArcGIS\\Projects\\KylesProProject3\\Profiles.gdb\\SurfaceMZ'>

In [13]:
#add the sewer lines as events for labeling the diameter, material, slope, etc

arcpy.management.CalculateField("ProfleLine", "StartM", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("ProfleLine", "EndM", "!Shape_Length!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.CreateRoutes("ProfleLine", "MAINTYPE", ws+r"\ProfleLineRoute", "TWO_FIELDS", "StartM", "EndM", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.lr.MakeRouteEventLayer("ProfleLineRoute", "MAINTYPE", "profileSegments", "Main Type; Line; FMEAS; TMEAS", "profileSegmentsEvents", None, "NO_ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")


<Result 'profileSegmentsEvents'>

In [4]:
#add the sewer lines as events for labeling the diameter, material, slope, etc

arcpy.management.CalculateField("ProfleLine", "StartM", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("ProfleLine", "EndM", "!Shape_Length!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.CreateRoutes("ProfleLine", "MAINTYPE", ws+r"\ProfleLineRoute", "TWO_FIELDS", "StartM", "EndM", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")
arcpy.lr.MakeRouteEventLayer("ProfleLineRoute", "MAINTYPE", "profileSegments", "Main Type; Line; FMEAS; TMEAS", "profileSegmentsEvents", None, "NO_ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")


<Result 'profileSegmentsEvents'>

In [5]:
#make better routes that are on the basis of the conduits and that gap across the drops

arcpy.management.CalculateField("WestdaleSewerProfile", "StartM", "0", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("WestdaleSewerProfile", "EndM", "!Shape_Length!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.CreateRoutes("WestdaleSewerProfile", "FACILITYID", ws+r"\SewerRoute", "TWO_FIELDS", "StartM", "EndM", "UPPER_LEFT", 1, 0, "IGNORE", "INDEX")

<Result '\\\\citydata\\users\\kgonterwitz\\projectreviews\\westdale\\WestdaleSewerDesignFile.gdb\\SewerRoute'>

In [6]:

arcpy.management.CalculateField("WestdaleSewerProfile", "PipeStartM", "2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("WestdaleSewerProfile", "PipeEndM", "!Shape_Length!-2", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.lr.LocateFeaturesAlongRoutes("WestdaleSewerProfile", "SewerRoute", "FACILITYID", "0 Feet", ws+r"\ConduitSegs", "RID; Line; FMEAS; TMEAS", "FIRST", "DISTANCE", "ZERO", "FIELDS", "M_DIRECTON")
arcpy.lr.MakeRouteEventLayer("SewerRoute", "FACILITYID", "ConduitSegs", "Facility Identifier; Line; PipeStartM; PipeEndM", "ConduitSegEvents", None, "ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")
arcpy.lr.MakeRouteEventLayer("SewerRoute", "FACILITYID", "ConduitSegs", "Facility Identifier; Line; FMEAS; TOMEAS", "FlowlineEvents", None, "ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")


<Result 'ConduitSegEvents'>

In [9]:
arcpy.lr.MakeRouteEventLayer("SewerRoute", "FACILITYID", "ConduitSegs", "Facility Identifier; Line; FMEAS; TMEAS", "FlowlineEvents", None, "ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")
arcpy.lr.MakeRouteEventLayer("SewerRoute", "FACILITYID", "ConduitSegs", "Facility Identifier; Line; FMEAS; TMEAS", "FlowlineEvents", None, "ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")

ExecuteError: ERROR 000211: Cannot create route event source
Failed to execute (MakeRouteEventLayer).


In [3]:
#check as-built data vs GIS data 
#adjust and correct as necessary
#use slope calc to pin drop structures

slope = 15.6/100

usfl = 975.06

length = 126

drop = length*slope

print(drop)

dsbyslope = usfl-drop
print(dsbyslope)

19.656
955.404


In [5]:

usout = 955
dsin = 945
length = 125

slope = (usout-dsin)/length*100
print(slope)

8.0
